## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-27-03-46-43 +0000,startribune,Twins stun Blue Jays 7-5 behind homers from Mi...,https://www.startribune.com/minnesota-twins-to...
1,2025-08-27-03-46-15 +0000,nypost,Some FEMA employees put on leave following sig...,https://nypost.com/2025/08/26/us-news/some-fem...
2,2025-08-27-03-28-49 +0000,nypost,Minnesota mom who called 5-year-old with autis...,https://nypost.com/2025/08/26/us-news/minnesot...
3,2025-08-27-03-20-31 +0000,startribune,GOP and DFL-endorsed candidates win special pr...,https://www.startribune.com/primary-elections-...
4,2025-08-27-03-10-08 +0000,bbc,Kpop Demon Hunters becomes Netflix's most view...,https://www.bbc.com/news/articles/c1kz12v08l1o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,62
403,cook,17
326,new,17
402,lisa,16
153,gaza,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...,174
84,2025-08-26-22-42-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,162
85,2025-08-26-22-33-58 +0000,nypost,Fed governor Lisa Cook hires Hunter Biden lawy...,https://nypost.com/2025/08/26/us-news/fed-gove...,150
202,2025-08-26-16-12-46 +0000,wapo,Federal Reserve governor Lisa Cook says she’ll...,https://www.washingtonpost.com/business/2025/0...,144
136,2025-08-26-20-04-00 +0000,wsj,President Trump has told advisers that he want...,https://www.wsj.com/politics/policy/trump-weig...,144


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,174,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...
35,53,2025-08-27-01-00-00 +0000,wsj,The first Black woman to serve as a Federal Re...,https://www.wsj.com/economy/central-banking/li...
237,51,2025-08-26-13-35-35 +0000,nypost,Netanyahu calls deadly double strike on Gaza h...,https://nypost.com/2025/08/26/world-news/netan...
254,47,2025-08-26-10-39-53 +0000,cbc,Australia accuses Iran of being behind two ant...,https://www.cbc.ca/news/world/australia-iran-a...
248,47,2025-08-26-11-52-00 +0000,wsj,Illinois Democratic Gov. JB Pritzker promised ...,https://www.wsj.com/politics/policy/chicago-vo...
99,40,2025-08-26-21-51-00 +0000,wsj,Canada’s manufacturers and wholesalers look to...,https://www.wsj.com/articles/canada-factory-wh...
145,38,2025-08-26-19-28-36 +0000,nypost,Cracker Barrel ripped as viral ‘Uncle Herschel...,https://nypost.com/2025/08/26/business/cracker...
21,37,2025-08-27-01-59-36 +0000,nyt,SpaceX Starship Completes Successful Test Flig...,https://www.nytimes.com/2025/08/26/science/spa...
141,35,2025-08-26-19-43-01 +0000,nypost,Trump insists Russia refusal to meet Ukraine’s...,https://nypost.com/2025/08/26/us-news/trump-in...
15,34,2025-08-27-02-18-27 +0000,nypost,Wisconsin judge accused of helping man dodge I...,https://nypost.com/2025/08/26/us-news/wisconsi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
